In [8]:
import os
from dotenv import load_dotenv
import openai
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.tools import AIPluginTool
from langchain.agents import load_tools, initialize_agent, AgentType, mrkl

load_dotenv()
langchain.debug = True

In [10]:
# 基盤モデルのセットアップ
openai.api_type = "azure"
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")

llm = ChatOpenAI(
    model_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
    model_kwargs={
        'deployment_id': os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    },
    temperature=0,
)

# テスト実行
llm.predict("Hello, I'm a customer support representative.", max_tokens=100)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Hello, I'm a customer support representative."
  ]
}
[llm/end] [1:llm:ChatOpenAI] [9.17s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Hello! How can I assist you today?",
        "generation_info": {
          "finish_reason": "stop"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Hello! How can I assist you today?",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 9,
      "prompt_tokens": 16,
      "total_tokens": 25
    },
    "model_name": "gpt-35-turbo"
  },
  "run": null
}


'Hello! How can I assist you today?'

In [16]:
# HTTPリクエストを行うためのツールをロード
tools = []
tools.append(load_tools(["requests"])[0])
# tools.append(load_tools(["requests_post"])[0])

# プラグインのロード
plugin_url = "http://localhost:5000/.well-known/ai-plugin.json"
tools.append(AIPluginTool.from_plugin_url(plugin_url))

tools

[RequestsGetTool(name='requests_get', description='A portal to the internet. Use this when you need to get specific content from a website. Input should be a  url (i.e. https://www.google.com). The output will be the text response of the GET request.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, requests_wrapper=TextRequestsWrapper(headers=None, aiosession=None, auth=None)),
 AIPluginTool(name='todo', description='Call this tool to get the OpenAPI spec (and usage guide) for interacting with the TODO List API. You should only call this ONCE! What is the TODO List API useful for? Manage your TODO list. You can add, remove and view your TODOs.', args_schema=<class 'langchain.tools.plugin.AIPluginToolSchema'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, plugin=AIPlugin(schema_version='v1', name_for_model='todo'

In [17]:
SUFFIX = """
'Answer should be in Japanese. Use http instead of https for endpoint.
"""

# エージェントのセットアップ
agent_chain = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    vebose=True,
)

# エージェン実行
result = agent_chain.run("現在のtodoを表示して")
print(result)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "現在のtodoを表示して"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "現在のtodoを表示して",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Answer the following questions as best you can. You have access to the following tools:\n\nrequests_get: A portal to the internet. Use this when you need to get specific content from a website. Input should be a  url (i.e. https://www.google.com). The output will be the text response of the GET request.\ntodo: Call this tool to get the OpenAPI spec (and usage guide) for interacting with the TODO List API. You should only call this ONCE! What is the TODO List API useful for? Manage your TODO list. You can add, remove and view your TODOs.\n\nUse the following